# Análisis no supervisado de zonas geográficas utilizando el dataset Covertype: identificación de patrones ambientales para estudios climáticos

**Participantes:** Raúl Urzúa, Adán Marchena
**Profesor guía:** Ronal Manuel Coronado
**Dominio de datos:** Tabular

## Definición del problema y objetivos

### Definición del problema

El dataset Covertype contiene información topográfica y ecológica de distintas regiones forestales, incluyendo variables como elevación, pendiente y distancias a cuerpos de agua. Analizar estos datos mediante técnicas de aprendizaje no supervisado permite identificar zonas con características ambientales similares sin depender de etiquetas predefinidas.
Esta clasificación es relevante porque facilita el estudio de la distribución ecológica y sus variaciones, lo cual puede emplearse como base para monitorear cambios ambientales, gestionar recursos naturales y comprender mejor los efectos del cambio climático sobre distintos tipos de terreno y vegetación.

### Justificación del enfoque no supervisado (PCA, clustering)

El aprendizaje no supervisado es adecuado para este problema porque no se dispone de una clasificación previa de las zonas. Métodos como el Análisis de Componentes Principales (PCA) permiten reducir la dimensionalidad del conjunto de datos y visualizar las relaciones entre variables, mientras que los algoritmos de clustering (como K-means o DBSCAN) ayudan a descubrir agrupaciones naturales de regiones con características similares.

### Objetivos específicos

- Identificar patrones o grupos de regiones con características ambientales semejantes.
- Aplicar reducción de dimensionalidad mediante PCA para mejorar la interpretación de los datos.
- Evaluar diferentes métodos de clustering y comparar su capacidad de distinguir zonas ecológicas.
- Sentar una base de análisis para estudios futuros sobre variaciones climáticas y ecológicas.

## Recolección y preparación de datos

### Fuente de datos

El dataset Covertype fue obtenido desde la plataforma Hugging Face Datasets, en la ruta:
hf://datasets/polinaeterna/tabular-benchmark/clf_cat/covertype.csv.

Este conjunto de datos contiene 423.680 registros y 55 columnas, incluyendo una variable objetivo (Cover_Type) y 54 variables predictoras numéricas y categóricas.
Las variables describen distintas características geográficas y ecológicas de regiones forestales, como elevación, pendiente, orientación, distancias a cuerpos de agua, tipo de suelo y otros indicadores ambientales.

Aunque originalmente el dataset fue diseñado para clasificar tipos de cobertura forestal, en este trabajo se utiliza con un enfoque no supervisado, con el objetivo de agrupar regiones con condiciones ecológicas similares y facilitar el análisis de patrones ambientales en futuras investigaciones sobre variabilidad climática.

### Preprocesamiento aplicado

### Preprocesamiento aplicado (limpieza, normalización, transformación, etc.)

Se realizó el siguiente pipeline de inspección y preprocesamiento exploratorio:

1. **Importación y revisión inicial**
- Importación de librerías (pandas, numpy, sklearn, matplotlib/seaborn, etc.) y carga del fichero `covertype.csv`.
- Revisión de la estructura con `df.shape`, `df.info()` y `df.describe().T` para entender dimensiones, tipos y estadísticas básicas.

2. **Calidad de los datos**
- Búsqueda de valores faltantes (`isnull().sum()`) — no se detectaron nulos.
- Comprobación de duplicados (`duplicated().sum()`) — no se detectaron registros duplicados.

3. **Exploración univariante y bivariante**
- Histogramas / plots de densidad para variables continuas seleccionadas: `Elevation`, `Slope`, `Horizontal_Distance_To_Hydrology`, `Hillshade_Noon`.
- Gráfico de distribución de la variable class (frecuencias relativas) para observar balance de clases.
- Heatmap de correlación entre variables numéricas reales para detectar relaciones lineales y posibles multicolinealidades.

4. **Clasificación de variables**
- Identificación de la estructura del dataset: 423 680 observaciones y 55 columnas (10 continuas, 4 binarias de “Wilderness Area”, 40 binarias de “Soil Type”, 1 variable class).
- Las variables binarias se mantienen como tales (0/1). Las variables categóricas ya se encuentran en formato binario (one-hot-like) según la estructura del dataset.

5. **Escalado**
- Aplicación de `StandardScaler` a las variables continuas para llevarlas a media 0 y varianza 1 (necesario para PCA y algoritmos sensibles a escala como K-Means y GMM).

6. **Reducción de dimensionalidad (planificada)**
- Se planifica aplicar PCA para: (a) reducir dimensionalidad y ruido, (b) facilitar visualización (2D/3D) y (c) alimentar clustering si se desea. (Ver sección de metodología técnica para detalles de n.º de componentes).

7. **Preparación final del dataset para clustering**
- Construcción de la matriz de características X escalada (contínuas escaladas + variables binarias tal cual).
- Separación opcional de la columna `class` para análisis complementario (no se usa para entrenar los modelos no supervisados).

### Decisiones tomadas y razones

**Documentar la fuente exacta:** Se especificó la ruta en Hugging Face (hf://datasets/polinaeterna/tabular-benchmark/clf_cat/covertype.csv) y la estructura real (423 680 × 55) para reproducibilidad.

**No imputación ni eliminación de filas:** Al no existir valores nulos ni duplicados, no se realizó imputación ni poda de registros, manteniendo la máxima información original.

**No eliminar variables por correlación:** El heatmap mostró correlaciones moderadas pero no lo suficientemente altas para justificar eliminación automática; se decidió conservar todas las variables para que PCA y los modelos de clustering determinen la relevancia.

**Escalado con StandardScaler:** Elegido porque K-Means, GMM y PCA son sensibles a la escala de las variables continuas; el escalado evita que variables con magnitudes mayores dominen las distancias.

**Mantener variables binarias sin transformar:** Las columnas binarias (Wilderness Area, Soil Type) se mantienen en 0/1 porque representan presencia/ausencia y son interpretables por los algoritmos de clustering (si se prefiriera, en análisis alternativo se podría usar una métrica mixta).

**PCA como herramienta de soporte (no como único paso):** Se usará PCA para visualización y para reducir dimensionalidad antes de algunos experimentos, pero se compararán resultados de clustering tanto sobre el espacio original escalado como sobre el espacio reducido por PCA para verificar robustez.

**Validación interna de clusters:** Se planificó evaluar calidad de agrupamientos con índices (silhouette, Davies–Bouldin) y análisis cualitativo (examinar centroides o perfiles promedio de cada cluster) para interpretar económicamente/ambientalmente los grupos.

### Resumen

El dataset fue inspeccionado, no presenta nulos ni duplicados y está estructurado en 423 680 observaciones por 55 columnas (10 continuas, 44 binarias incluyendo class). Se aplicó escalado estandarizado a las variables continuas y se mantuvieron las binarias sin cambios. Se conservaron todas las variables tras revisar correlaciones moderadas. Posteriormente se aplicará PCA para reducción/visualización y se probarán distintos algoritmos de clustering (K-Means, DBSCAN, GMM), comparando resultados sobre el espacio original escalado y sobre el espacio reducido por PCA.

## Metodología técnica: PCA y Clustering

### Aplicación de PCA (componentes, criterios, visualizaciones).

Para reducir la dimensionalidad del conjunto de datos y facilitar la visualización de las estructuras latentes, se aplicó Análisis de Componentes Principales (PCA).
El PCA permite transformar las variables originales en un conjunto de componentes ortogonales que capturan la mayor parte de la varianza total de los datos.

Se empleó el criterio de varianza acumulada ≥ 95 % para determinar el número óptimo de componentes, lo que resultó en 39 componentes principales.
Esta reducción permitió conservar la mayor parte de la información estadística eliminando redundancias y ruido.

Se generaron gráficos bivariados con las dos primeras componentes principales para observar posibles agrupamientos y la estructura general de los datos, lo que sirvió como base para aplicar los algoritmos de clustering posteriores.

### Algoritmos de clustering aplicados (k-means, DBSCAN, etc.) y parámetros utilizados.

Se probaron distintos algoritmos de agrupamiento con el objetivo de comparar su desempeño frente a la estructura de los datos:

**K-Means:**
- Parámetros principales: número de clusters k=2 (estimado mediante el método del codo y el coeficiente de Silhouette).
- Observación: el algoritmo logró una partición razonable, aunque sensible a la inicialización y al valor de k.

**DBSCAN:**
- Parámetros: eps=0.5, min_samples=10.
- Resultó ineficiente computacionalmente para el tamaño del conjunto de datos y no logró converger en equipos con recursos limitados.

**HDBSCAN:**
- Se utilizó HDBSCAN como alternativa a DBSCAN por su robustez frente a densidades variables y su menor necesidad de ajustar eps. 
- El algoritmo se ejecutó con min_cluster_size=10 (y min_samples en su valor por defecto), detectando múltiples regiones de alta densidad y clasificando puntos dispersos como ruido. Esta configuración permite identificar clusters relativamente pequeños, lo que fue útil para detectar subestructuras locales en el conjunto de datos sin imponer un número de clusters a priori.

**Gaussian Mixture Model (GMM):**
- Se ajustaron modelos con distintos números de componentes (entre 2 y 10) y se seleccionó el óptimo según el criterio BIC.
- El GMM permitió identificar clusters con formas elípticas y solapamientos, estimando probabilidades de pertenencia para cada punto.

### Evaluación de clusters / calidad (Silhouette, Davies–Bouldin, etc.)

Para evaluar la calidad de las particiones obtenidas, se emplearon métricas internas:

**Coeficiente de Silhouette:**
- Calcula la cohesión y separación de los clusters.
- Se obtuvieron valores entre 0.25 y 0.45, lo que indica una estructura de agrupamiento razonable pero con cierto solapamiento entre grupos.


## Análisis e interpretación de resultados

### Relación entre resultados y la tarea planteada

El objetivo del análisis fue identificar patrones latentes en los datos mediante métodos no supervisados, con el fin de explorar estructuras internas y potenciales agrupamientos naturales. Los algoritmos K-Means, HDBSCAN y Gaussian Mixture Model (GMM), aplicados sobre los componentes principales obtenidos con PCA, permitieron contrastar diferentes enfoques de clustering y evaluar su capacidad para capturar la complejidad de los datos. Los resultados muestran diferencias significativas entre métodos basados en particiones fijas (K-Means) y métodos basados en densidad o modelos probabilísticos (HDBSCAN y GMM), lo que evidencia la necesidad de técnicas más flexibles para datos con distribuciones no lineales y regiones de densidad variable.

### Insights o patrones descubiertos

- **K-Means** generó una partición clara en dos clusters principales, reflejando una separación macroestructural. No obstante, la forma no lineal de los datos y su distribución irregular sugiere que el algoritmo fuerza una estructura más simple de la que realmente existe.
- **HDBSCAN** identificó múltiples regiones de alta densidad y clasificó un número considerable de puntos como ruido. Este resultado indica que los datos presentan subestructuras internas y zonas de baja densidad que K-Means no captura. La detección de ruido sugiere la presencia de outliers o variabilidad compleja.
- **GMM** produjo una segmentación más suave y probabilística, permitiendo observar transiciones graduales entre grupos. Esto indica que parte de los datos presenta fronteras difusas entre clusters, lo cual concuerda con la distribución continua observada en el espacio PCA.

En conjunto, los resultados sugieren que la estructura de los datos no es completamente separable mediante límites lineales o particiones rígidas, y que existen múltiples regiones con diferente densidad y posible solapamiento entre grupos.

### Conclusiones principales

- El uso de PCA permitió reducir la dimensionalidad preservando la variabilidad esencial y facilitó la interpretación visual de los clusters.
- K-Means fue útil para obtener una estructura general, pero mostró limitaciones al suponer clusters convexos y de tamaño similar.
- HDBSCAN resultó más adecuado para capturar regiones complejas y outliers, revelando patrones locales y ruido estructural.
- GMM ofreció una visión probabilística coherente con datos con fronteras difusas, mostrando una transición gradual entre grupos.

En conjunto, los resultados sugieren que los datos presentan una estructura compleja, continua y con variaciones de densidad, por lo que métodos basados en densidad o mixturas gaussianas son más apropiados que particiones rígidas.